In [1]:
import pandas as pd
import numpy as np


NUM_EPISODES = 1000
MAX_EPISODE_LENGTH = 500

DEFAULT_DISCOUNT = 0.9
EPSILON = 0.5
LEARNINGRATE = 0.1
class QLearner(): 

    def __init__(self, num_states, num_actions, discount=DEFAULT_DISCOUNT, learning_rate=LEARNINGRATE): # You can add more arguments if you want
        self.name = "agent1"
        self.actions = list(range(num_actions))   # 0, 1, 2, 3, index
        print(self.actions)
        self.states = num_states
        self.discount = discount
        self.lr = learning_rate
        self.q_table = pd.DataFrame(columns = self.actions, dtype = np.float64) # create the q tab

    def process_experience(self, state, action, next_state, reward, done): # You can add more arguments if you want
        """
        Update the Q-value based on the state, action, next state and reward.
        """
        self.check_state_exist(next_state)
        q_predict = self.q_table.loc[state, action]
        
        if next_state != 'terminal':
            self.q_table.loc[state, action] = reward + self.discount * (np.sum(0.8 * () + 0.2 * ()))
#             q_target = reward + self.discount * self.q_table.loc[next_state, :].max()
        else :
#             q_target = reward
            
        self.q_table.loc[state, action] = (1 - self.lr) * self.q_table.loc[state, action] + self.lr * q_target

        pass

    def select_action(self, state, turns): # You can add more arguments if you want
        """
        Returns an action, selected based on the current state
        """
        self.check_state_exist(state)
        
        # if np.random.uniform() > (0.5 - 0.00045*turns):
        if np.random.uniform() > EPISON:
            state_action = self.q_table.loc[state, :]
            action = np.random.choice(state_action[state_action == np.max(state_action)].index)
        else :
            action = np.random.choice(self.actions)
        
        return action
        pass

    def check_state_exist(self, state):
        if state not in self.q_table.index:
            self.q_table = self.q_table.append(
                pd.Series(     # index and value
                    [0]*len(self.actions),     #data
                    index = self.q_table.columns, # index
                    name = state, #dtype
                )
                
            )
#             print (self.q_table)

    def report(self):
        """
        Function to print useful information, printed during the main loop
        """
        # write the optimal solurion
        print("---")


IndentationError: expected an indented block (<ipython-input-1-55781560b6c9>, line 35)

In [8]:
import sys
from contextlib import closing

import numpy as np
from six import StringIO, b

from gym import utils
from gym.envs.toy_text import discrete

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

MAPS = {
    "theAlley": [
        "S...H...H...G"
    ],
    "walkInThePark": [
        "S.......",
        ".....H..",
        "........",
        "......H.",
        "........",
        "...H...G"
    ],
    "1Dtest": [

    ],
    "4x4": [
        "S...",
        ".H.H",
        "...H",
        "H..G"
    ],
    "8x8": [
        "S.......",
        "........",
        "...H....",
        ".....H..",
        "...H....",
        ".HH...H.",
        ".H..H.H.",
        "...H...G"
    ],
}

POTHOLE_PROB = 0.2
BROKEN_LEG_PENALTY = -10
SLEEP_DEPRIVATION_PENALTY = -0.0
REWARD = 10

def generate_random_map(size=8, p=0.8):
    """Generates a random valid map (one that has a path from start to goal)
    :param size: size of each side of the grid
    :param p: probability that a tile is frozen
    """
    valid = False

    # DFS to check that it's a valid path.
    def is_valid(res):
        frontier, discovered = [], set()
        frontier.append((0,0))
        while frontier:
            r, c = frontier.pop()
            if not (r,c) in discovered:
                discovered.add((r,c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == 'G':
                        return True
                    if (res[r_new][c_new] not in '#H'):
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(['.', 'H'], (size, size), p=[p, 1-p])
        res[0][0] = 'S'
        res[-1][-1] = 'G'
        valid = is_valid(res)
    return ["".join(x) for x in res]


class DrunkenWalkEnv(discrete.DiscreteEnv):
    """
    A simple grid environment, completely based on the code of 'FrozenLake', credits to 
    the original authors.

    You're finding your way home (G) after a great party which was happening at (S).
    Unfortunately, due to recreational intoxication you find yourself only moving into 
    the intended direction 80% of the time, and perpendicular to that the other 20%.

    To make matters worse, the local community has been cutting the budgets for pavement
    maintenance, which means that the way to home is full of potholes, which are very likely
    to make you trip. If you fall, you are obviously magically transported back to the party, 
    without getting some of that hard-earned sleep.

        S...
        .H.H
        ...H
        H..G
    H : pothole, you have a POTHOLE_PROB chance of tripping
    G : goal, time for bed

    The episode ends when you reach the goal or trip.
    You receive a reward of +10 if you reach the goal, 
    but get a SLEEP_DEPRIVATION_PENALTY and otherwise.  剥夺睡眠

    """

    metadata = {'render.modes': ['human', 'ansi']}  # the meaning of ansi

    def __init__(self, desc=None, map_name="4x4",is_slippery=True):
        """ This generates a map and sets all transition probabilities.

            (by passing constructed nS, nA, P, isd to DiscreteEnv)
        """
        if desc is None and map_name is None:
            desc = generate_random_map()
        elif desc is None:
            desc = MAPS[map_name]

        self.desc = desc = np.asarray(desc,dtype='c')
        self.nrow, self.ncol = nrow, ncol = desc.shape  # start 6*8
        self.reward_range = (0, 1)

        nA = 4
        nS = nrow * ncol
        # compare desc with S, find the big S position
        isd = np.array(desc == b'S').astype('float64').ravel() # ravel: transfer n-D to 1-D 
#         print(isd)
# [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
#  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
# ---episode 0---
        isd /= isd.sum()

        # We need to pass 'P' to DiscreteEnv:
        # P dictionary dict of dicts of lists, where
        # P[s][a] == [(probability, nextstate, reward, done), ...]
        P = {s : {a : [] for a in range(nA)} for s in range(nS)}  # how can P represent so many params

        def to_s(row, col):
            return row*ncol + col

        #def inc(row, col, a):
        def intended_destination(row, col, a):
            if a == LEFT:
                col = max(col-1,0)
            elif a == DOWN:
                row = min(row+1,nrow-1)
            elif a == RIGHT:
                col = min(col+1,ncol-1)
            elif a == UP:
                row = max(row-1,0)
            return (row, col)

        def construct_transition_for_intended(row, col, a, prob, li):
            """ this constructs a transition to the "intended_destination(row, col, a)"
                and adds it to the transition list (which could be for a different action b).

            """
            newrow, newcol = intended_destination(row, col, a)  # the next step position
            newstate = to_s(newrow, newcol)   # how to understand the new state is a integer varaible
            newletter = desc[newrow, newcol]  # record the new letter
            done = bytes(newletter) in b'G'   # when home is done
            rew = REWARD if newletter == b'G' else SLEEP_DEPRIVATION_PENALTY
            li.append( (prob, newstate, rew, done) )


        for row in range(nrow):
            for col in range(ncol):
                # specify transitions for s=(row, col)
                s = to_s(row, col)
                letter = desc[row, col]
                for a in range(4):
                    # specify transitions for action a
                    li = P[s][a] # s is states, a is action
                    if letter in b'G':
                        #when we are at goal, we reset with prob. 1
                        li.append((1.0, s, 42, True))   # prob newstate reward done
                        #really, this should not be happening, since we get done
                        #when transitioning TO the goal state (not from it).
                    if letter in b'H':
                        #when we are at a pothole, we trip with prob. POTHOLE_PROB
                        li.append((POTHOLE_PROB, s, BROKEN_LEG_PENALTY, True))
                        construct_transition_for_intended(row, col, a, 1.0 - POTHOLE_PROB, li)
                        
                    else:
                        # with prob. 0.8 we move as intended:
                        construct_transition_for_intended(row, col, a, 0.8, li)
                        # but with prob. 0.1 we move sideways to intended:
                        for b in [(a-1)%4, (a+1)%4]: # 其他的  0.1
                            construct_transition_for_intended(row, col, b, 0.1, li)

        super(DrunkenWalkEnv, self).__init__(nS, nA, P, isd)

    def action_to_string(self, action_index):
        s ="{}".format(["Left","Down","Right","Up"][action_index])
        return s

    def render(self, mode='human'):
        outfile = StringIO() if mode == 'ansi' else sys.stdout

        row, col = self.s // self.ncol, self.s % self.ncol
        desc = self.desc.tolist()
        desc = [[c.decode('utf-8') for c in line] for line in desc]
        desc[row][col] = utils.colorize(desc[row][col], "red", highlight=True)
        if self.lastaction is not None:
            outfile.write(" (last action was '{action}')\n".format( action=self.action_to_string(self.lastaction) ) )
        else:
            outfile.write("\n")
        outfile.write("\n".join(''.join(line) for line in desc)+"\n")

        if mode != 'human':
            with closing(outfile):
                return outfile.getvalue()


In [9]:
import gym
DEFAULT_DISCOUNT = 0.9

def act_loop(env, agent, num_episodes):
    for episode in range(num_episodes):
        state = env.reset()         # state are observations

        print('---episode %d---' % episode)
        renderit = False            
        if episode % 1000 == 0:     # evert 10 times to update the visible environment
            renderit = True
        elif episode == 999:
            renderit = True

        for t in range(MAX_EPISODE_LENGTH):
            if renderit:
                env.render()     # update the visible environment
            printing=False
            if t % 500 == 499:
                printing = True   

            if printing:          # every 500 times print the stage ?
                print('---stage %d---' % t)
                agent.report()
                print("state:", state)

            action = agent.select_action(state, episode)    # choose the action
            new_state, reward, done, info = env.step(action)
            if printing:
                print("act:", action)
                print("reward=%s" % reward)

            agent.process_experience(state, action, new_state, reward, done)
            state = new_state
            if done:
                print("Episode finished after {} timesteps".format(t+1))
                env.render()
                agent.report()
                break

    env.close()


if __name__ == "__main__":
#     env = DrunkenWalkEnv(map_name="walkInThePark")   #make sure the enviroment
    env = DrunkenWalkEnv(map_name="theAlley")
    # env = simple_grid.DrunkenWalkEnv(map_name="theAlley")
    num_a = env.action_space.n   # how many actions
#     print(num_a)
    if (type(env.observation_space)  == gym.spaces.discrete.Discrete):
        num_o = env.observation_space.n   # how many observations
#         print(num_o)
    else:
        raise("Qtable only works for discrete observations")


    discount = DEFAULT_DISCOUNT
    ql = QLearner(num_o, num_a, discount) #<- QTable
#     print(ql.q_table)
    act_loop(env, ql, NUM_EPISODES)
    print(ql.q_table)



[0, 1, 2, 3]
---episode 0---

S...H...H...G
 (last action was 'Right')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Right')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Up')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Down')
S...H...H...G
 (last action was 'Down')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Down')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Down')
S...H...H...G
 (last action was 'Right')
S...H...H...G
 (last action was 'Up')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Down')
S...H...H...G
 (last action was 'Right')
S...H...H...G
 (last action was 'Down')
S...H...H...G
 (last action was 'Down')
S...H...H...G
 (last action was 'Right')
S...H...H...G
 (last action was 'Left')
S...H...H...G
 (last action was 'Down')
S...H...H

---stage 499---
---
state: 2
act: 3
reward=-0.0
---episode 19---
Episode finished after 167 timesteps
 (last action was 'Up')
S...H...H...G
---
---episode 20---
Episode finished after 230 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 21---
Episode finished after 122 timesteps
 (last action was 'Up')
S...H...H...G
---
---episode 22---
---stage 499---
---
state: 1
act: 1
reward=-0.0
---episode 23---
Episode finished after 175 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 24---
Episode finished after 337 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 25---
Episode finished after 21 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 26---
Episode finished after 89 timesteps
 (last action was 'Up')
S...H...H...G
---
---episode 27---
Episode finished after 37 timesteps
 (last action was 'Down')
S...H...H...G
---
---episode 28---
Episode finished after 223 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 29

Episode finished after 62 timesteps
 (last action was 'Up')
S...H...H...G
---
---episode 99---
Episode finished after 150 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 100---
Episode finished after 52 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 101---
Episode finished after 62 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 102---
Episode finished after 274 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 103---
Episode finished after 436 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 104---
Episode finished after 24 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 105---
Episode finished after 246 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 106---
Episode finished after 9 timesteps
 (last action was 'Up')
S...H...H...G
---
---episode 107---
Episode finished after 24 timesteps
 (last action was 'Up')
S...H...H...G
---
---episode 108---
Episode finished af

---stage 499---
---
state: 1
act: 3
reward=-0.0
---episode 181---
Episode finished after 335 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 182---
---stage 499---
---
state: 0
act: 2
reward=-0.0
---episode 183---
Episode finished after 164 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 184---
Episode finished after 74 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 185---
Episode finished after 186 timesteps
 (last action was 'Down')
S...H...H...G
---
---episode 186---
Episode finished after 136 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 187---
Episode finished after 69 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 188---
Episode finished after 140 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 189---
Episode finished after 70 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 190---
Episode finished after 77 timesteps
 (last action was 'Left')
S...H...H...G
-

Episode finished after 190 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 261---
Episode finished after 101 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 262---
Episode finished after 420 timesteps
 (last action was 'Down')
S...H...H...G
---
---episode 263---
Episode finished after 323 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 264---
Episode finished after 122 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 265---
Episode finished after 14 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 266---
Episode finished after 123 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 267---
Episode finished after 144 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 268---
Episode finished after 493 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 269---
Episode finished after 211 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 270---
Episode

...

Episode finished after 84 timesteps
 (last action was 'Up')
S...H...H...G
---
---episode 589---
Episode finished after 12 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 590---
Episode finished after 18 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 591---
Episode finished after 21 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 592---
Episode finished after 45 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 593---
Episode finished after 17 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 594---
Episode finished after 8 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 595---
Episode finished after 15 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 596---
Episode finished after 28 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 597---
Episode finished after 27 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 598---
Episode finish

Episode finished after 363 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 666---
Episode finished after 325 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 667---
---stage 499---
---
state: 2
act: 2
reward=-0.0
---episode 668---
Episode finished after 157 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 669---
---stage 499---
---
state: 0
act: 3
reward=-0.0
---episode 670---
Episode finished after 260 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 671---
Episode finished after 83 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 672---
Episode finished after 139 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 673---
Episode finished after 225 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 674---
Episode finished after 55 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 675---
---stage 499---
---
state: 2
act: 1
reward=-0.0
---episode 676---
Epis

---episode 747---
Episode finished after 25 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 748---
Episode finished after 38 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 749---
Episode finished after 80 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 750---
Episode finished after 30 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 751---
Episode finished after 185 timesteps
 (last action was 'Down')
S...H...H...G
---
---episode 752---
Episode finished after 250 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 753---
Episode finished after 449 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 754---
Episode finished after 45 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 755---
Episode finished after 86 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 756---
Episode finished after 54 timesteps
 (last action was 'Right')
S...H...H...G
---
---episo

---stage 499---
---
state: 1
act: 0
reward=-0.0
---episode 832---
Episode finished after 160 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 833---
Episode finished after 290 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 834---
---stage 499---
---
state: 0
act: 3
reward=-0.0
---episode 835---
---stage 499---
---
state: 2
act: 1
reward=-0.0
---episode 836---
Episode finished after 104 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 837---
Episode finished after 136 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 838---
Episode finished after 185 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 839---
---stage 499---
---
state: 1
act: 2
reward=-0.0
---episode 840---
---stage 499---
---
state: 1
act: 3
reward=-0.0
---episode 841---
---stage 499---
---
state: 1
act: 1
reward=-0.0
---episode 842---
Episode finished after 423 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 843---
Epis

---
---episode 927---
Episode finished after 5 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 928---
Episode finished after 6 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 929---
Episode finished after 20 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 930---
Episode finished after 5 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 931---
Episode finished after 124 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 932---
---stage 499---
---
state: 2
act: 3
reward=-0.0
---episode 933---
Episode finished after 92 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 934---
Episode finished after 40 timesteps
 (last action was 'Left')
S...H...H...G
---
---episode 935---
Episode finished after 437 timesteps
 (last action was 'Right')
S...H...H...G
---
---episode 936---
---stage 499---
---
state: 0
act: 2
reward=-0.0
---episode 937---
Episode finished after 378 timesteps
 (last action was '

# Question 1

After running the python code to execte in these two environment("WalkInThePark" and "theAlley"), 
It is clear see that "theAlley" is more difficult to learn in. Since it don`t have a path can be go through.

# Question 2

How to ajust the optimal, anyway it definitely not be the optimal

It is optimal solution, Episode finished after 12 timesteps. And the report is : R R R R R R R D R D R R.
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Down')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Down')
S.......
.....H..
........
......H.
........
...H...G
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
Episode finished after 12 timesteps
 (last action was 'Right')
S.......
.....H..
........
......H.
........
...H...G
---

# Question 3 

safaefag

sdafeag